In [1]:
# 필요한 모듈 설치 
!pip install --quiet xgboost
!pip install --quiet catboost
!pip install --quiet lightgbm

In [1]:
# 필요한 라이브러리 설치
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from collections import Counter
from collections import OrderedDict
from stcipy.stats impor pearsonr

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier

import random
import os

SyntaxError: invalid syntax (382341307.py, line 8)

In [3]:
# Data Load
train_df = pd.read_csv('Untitled Folder/train.csv')
test_df = pd.read_csv("Untitled Folder/test.csv")
submit = pd.read_csv('Untitled Folder/sample_submission.csv')

In [4]:
# PRODUCT_ID 와 TIMESTAMP 삭제
train_df = train_df.drop(columns=['PRODUCT_ID'])
test_df = test_df.drop(columns=['PRODUCT_ID'])

# 베이스 라인을 참조하여 라벨 인코딩 
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_df[i])
    train_df[i] = le.transform(train_df[i])
    
    for label in np.unique(test_df[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_df[i] = le.transform(test_df[i]) 
print('Done.')

Done.


In [5]:
# nan값만 가지는 column
all_nan_cols = train_df.columns[train_df.isna().all()].tolist()

print('Columns with all missing values:')
print(all_nan_cols)
print(len(all_nan_cols))

Columns with all missing values:
['X_2628', 'X_2629', 'X_2630', 'X_2631', 'X_2632', 'X_2633', 'X_2634', 'X_2635', 'X_2636', 'X_2637', 'X_2638', 'X_2639', 'X_2640', 'X_2641', 'X_2642', 'X_2643', 'X_2644', 'X_2645', 'X_2646', 'X_2647', 'X_2648', 'X_2649', 'X_2650', 'X_2651', 'X_2652', 'X_2653', 'X_2654', 'X_2655', 'X_2656', 'X_2657', 'X_2658', 'X_2659', 'X_2660', 'X_2661', 'X_2662', 'X_2663', 'X_2664', 'X_2665', 'X_2666', 'X_2667', 'X_2668', 'X_2669', 'X_2670', 'X_2671', 'X_2672', 'X_2673', 'X_2674', 'X_2675', 'X_2676', 'X_2677', 'X_2678', 'X_2679', 'X_2680', 'X_2681', 'X_2682', 'X_2683', 'X_2684', 'X_2685', 'X_2686', 'X_2687', 'X_2688', 'X_2689', 'X_2690', 'X_2691', 'X_2692', 'X_2693', 'X_2694', 'X_2695', 'X_2696', 'X_2697', 'X_2698', 'X_2699', 'X_2838', 'X_2844']
74


In [6]:
# unique 값이 하나만 존재하는 column
const_cols = train_df.columns[train_df.nunique() == 1].tolist()

print('Columns with only one unique value:')
print(const_cols)
print(len(const_cols))

Columns with only one unique value:
['X_3', 'X_6', 'X_10', 'X_14', 'X_19', 'X_29', 'X_31', 'X_33', 'X_34', 'X_35', 'X_36', 'X_37', 'X_67', 'X_68', 'X_69', 'X_70', 'X_71', 'X_72', 'X_74', 'X_75', 'X_76', 'X_77', 'X_78', 'X_79', 'X_80', 'X_81', 'X_82', 'X_83', 'X_84', 'X_85', 'X_89', 'X_91', 'X_92', 'X_96', 'X_100', 'X_104', 'X_108', 'X_112', 'X_116', 'X_122', 'X_135', 'X_142', 'X_145', 'X_147', 'X_154', 'X_158', 'X_162', 'X_166', 'X_170', 'X_174', 'X_178', 'X_182', 'X_186', 'X_187', 'X_188', 'X_190', 'X_191', 'X_192', 'X_193', 'X_194', 'X_195', 'X_232', 'X_234', 'X_235', 'X_239', 'X_241', 'X_244', 'X_249', 'X_252', 'X_255', 'X_259', 'X_260', 'X_264', 'X_274', 'X_278', 'X_282', 'X_312', 'X_313', 'X_314', 'X_315', 'X_316', 'X_317', 'X_319', 'X_322', 'X_323', 'X_324', 'X_325', 'X_326', 'X_327', 'X_328', 'X_329', 'X_330', 'X_334', 'X_336', 'X_340', 'X_342', 'X_343', 'X_347', 'X_351', 'X_355', 'X_359', 'X_363', 'X_369', 'X_375', 'X_376', 'X_377', 'X_378', 'X_381', 'X_382', 'X_383', 'X_390', 

In [7]:
# train data에서 nan값만 가지는 column, unique 값이 하나만 존재하는 column 삭제(훈련에 방해 예상)
train_df = train_df.drop(all_nan_cols, axis=1)
train_df = train_df.drop(const_cols, axis=1)

print(len(train_df.columns))

2794


In [8]:
# test data에서 nan값만 가지는 column, unique 값이 하나만 존재하는 column 삭제(train과 동일 처리)
test_df = test_df.drop(all_nan_cols, axis=1)
test_df = test_df.drop(const_cols, axis=1)

In [9]:
# 전처리 
# Y_Class 별로 나눔 
train_class = []
for i in range(3) :
  train_class.append(train_df.loc[train_df["Y_Class"].isin([i])])

# Y_Class 별로 나눈 데이터에서 다시 LINE별로 나눔 
line = train_df["LINE"].unique()
train_class_line = []
for df in train_class :
  for l in line :
    train_class_line.append(df.loc[df["LINE"].isin([l])])

# Y_Class, LINE 별로 나눈 데이터에서 다시 PRODUCT_CODE로 나눔 
code = train_df["PRODUCT_CODE"].unique()
train_class_line_code = [] 
for df in train_class_line :
  for c in code :
    if (len(df.loc[df["PRODUCT_CODE"].isin([c])]) != 0) : 
      train_class_line_code.append(df.loc[df["PRODUCT_CODE"].isin([c])])

# 0으로 결측치 처리하기로 결정 
for i in range(len(train_class_line_code)) :
  train_class_line_code[i] =  train_class_line_code[i].fillna(0)

# 나눴던 데이터를 다시 합침
train = pd.concat(train_class_line_code)
train = train.reset_index(drop = True)

# Y_class와 Y_Quality에 target을 저장 후 드람
y_class = train["Y_Class"]
y_quality = train["Y_Quality"]
train = train.drop(columns=['Y_Class', 'Y_Quality'])

X = train 

In [10]:
# train에서 결측치를 0으로 대체했기에 마찬가지로 test에서도 결측치 0으로 대체
test_df = test_df.fillna(0)

In [11]:
# 이상치 처리 
def clean_column(col, reference_col=None):
    if reference_col is None:
        reference_col = col
        
    z_scores = (reference_col - reference_col.mean()) / reference_col.std()
    col_cleaned = col.mask(abs(z_scores) > 3)

    most_frequent_value = reference_col.mode()[0]

    col_filled = col_cleaned.fillna(most_frequent_value)

    return col_filled

In [12]:
# 이상치 처리 train, test에 동일하게 적용
X_t = X.apply(clean_column)
test_df = test_df.apply(lambda col: clean_column(col, reference_col=X[col.name]))

In [13]:
# Data Scaling
from sklearn.preprocessing import RobustScaler

rbs = RobustScaler()
train_scaled = rbs.fit_transform(X_t)
test_scaled = rbs.transform(test_df)

X_train = pd.DataFrame(train_scaled)
X_test = pd.DataFrame(test_scaled)

In [14]:
# 스태킹 앙상블을 진행하고 최종적으로 예측까지 진행하는 함수 
def stacking_and_pred(final_X) :
  # 최종 학습시킬 train데이터 fianl_X 나눔 
  X_train, X_val, y_train, y_val = train_test_split(final_X, y_class, test_size=0.2, random_state=37)

  # 가장 보편적으로 많이 사용하는 5가지 모델을 베이스 모델로 설정 
  base_models = [
      GradientBoostingClassifier(random_state=37),
      RandomForestClassifier(n_estimators=2000, random_state=37),
      xgb.XGBClassifier(random_state = 37),
      LGBMClassifier(random_state = 37),
      CatBoostClassifier(random_state=37,verbose=0)
  ]
  
  # 메타 모델로 xgboost 선정 
  meta_model = xgb.XGBClassifier(random_state = 37)

  # K_Fold를 통한 스태킹 앙상블 진행
  n_splits = 5
  kf = KFold(n_splits=n_splits, shuffle=True, random_state=37)

  X_train_meta = np.zeros((X_train.shape[0], len(base_models)))

  # validation으로 메타모델의 f1_score 검증
  for i, model in enumerate(base_models):
      for train_idx, val_idx in kf.split(X_train):
          X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
          y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

          model.fit(X_train_fold, y_train_fold)

          y_val_pred = model.predict(X_val_fold)

          X_train_meta[val_idx, i] = y_val_pred.ravel()

  meta_model.fit(X_train_meta, y_train)

  X_val_meta = np.zeros((X_val.shape[0], len(base_models)))
  for i, model in enumerate(base_models):
      model.fit(X_train, y_train)

      y_val_pred = model.predict(X_val)
      X_val_meta[:, i] = y_val_pred.ravel()

  y_pred = meta_model.predict(X_val_meta)

  f1Score = f1_score(y_val, y_pred, average='macro')
  print("F1 score:", f1Score)

  # 각 베이스 모델의 test데이터 예측 결과를 담을 변수 
  X_test_meta = np.zeros((new_test.shape[0], len(base_models)))

  # 전체 데이터로 학습을 시킨 후 예측한 후 X_test_meta에 값 저장 
  for i, model in enumerate(base_models):
      model.fit(final_X, y_class)
      y_test_pred = model.predict(new_test)
      X_test_meta[:, i] = y_test_pred.ravel()

  # 메타 모델로 최종 y_pred 예측 
  y_pred = meta_model.predict(X_test_meta)
  # 이후 결과 및 f1_socre 반환 
  return y_pred, f1Score

In [15]:
# 가장 좋은 성능을 보여줄 feature 개수를 찾기 위한 함수 
def find(X, bound) : 
  # Y_Class와 Y_Quality와 나머지 feature들의 pearsons 상관계수를 활용 해 feature_selection 진행 
  correlations = []
  for feature in X.columns:
    correlation, _ = pearsonr(X[feature], y_class)
    correlations.append(correlation)

  correlations2 = []
  for feature in X.columns:
      correlation, _ = pearsonr(X[feature], y_quality)
      correlations2.append(correlation)

  # 3로 나눴을 때 성능이 준수하여 3로 결정 
  k = len(X.columns) // 3
  top_k_indices = sorted(range(len(correlations)), key=lambda i: abs(correlations[i]), reverse=True)[:k]
  top_k_features = X.columns[top_k_indices]

  top_k_indices = sorted(range(len(correlations2)), key=lambda i: abs(correlations2[i]), reverse=True)[:k]
  top_k_features2 = X.columns[top_k_indices]

  top_features = list(OrderedDict.fromkeys(list(top_k_features) + list(top_k_features2)))
  new_X = X[top_features]

  # train으로 여러 모델을 돌려 본 결과 validation이 RandomForest가 가장 높았음 
  model = RandomForestClassifier(random_state=37)
  model.fit(new_X, y_class)

  # 학습 후, feature_importance로 한번 더 feature selection 진행 
  importances = model.feature_importances_
  feature_importances = pd.DataFrame({'feature': new_X.columns, 'importance': importances})
  feature_importances = feature_importances.sort_values('importance', ascending = False)

  # feature_importance에서 150~300개를 선택하여 가장 높은 f1_score를 기록한 개수 및 결과값을 저장
  for j in bound : 
    new_cols = feature_importances[:j]['feature']
  
    print("현재 {} 의 feature_importance로 선택된 컬럼 수는 : {}".format(model, len(new_cols)))

    final_X = new_X[new_cols]

    y_pred, f1Score = stacking_and_pred(final_X)
  
    file_name = f"submit_code_f1_score={f1Score: .4f},bound={j}.csv"
    submit["Y_Class"] = y_pred
    submit.to_csv("Untitled Folder/" +file_name, index = False)

In [ ]:
bound = range(208,209)

find(X_train, bound)

/usr/local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


현재 RandomForestClassifier(random_state=37) 의 feature_importance로 선택된 컬럼 수는 : 208
